<a href="https://colab.research.google.com/github/shenith084/Data-Science-Report/blob/main/Data_Science_Report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Assignment - 01 End-to-End Data Science**
# **Report**

##Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns